<br>

# CAO Points Analysis

In [1]:
# Importing the requests package so I can make HTTP requests
import requests as rq

#Import regualar expresions. This will match strings to patterns later on.
import re

#Import date and times. Allows us to time stamp files.
import datetime as dt

#importing Pandas for working with data.
import pandas as pd

#import url lib for downloading.
import urllib.request as urlrq

print("done")

done


In [ ]:
#getting the current date and time formated as a string.
now = dt.datetime.now()

#format as a string
nowstr = now.strftime('%Y%m%d_%H%M%S')

<br>


## 2021 Points


http://www.cao.ie/index.php?page=points&p=2021

In [2]:
#fetches the cao points url
resp = rq.get('http://www2.cao.ie/points/l8.php')
#if the response is 200 everything is working ok so far. If it says 404 it is not ok.
resp

<Response [200]>

## Saving the original data set

In [4]:
#creating a file path for original data.
path = 'data/cao2021_' + nowstr + '.html'

In [5]:
# fixing the encoding. The server on cao website uses the wrong encoding so changing it to cp1252.
original_encoding = resp.encoding

resp.encoding = 'cp1252'

In [6]:
#save the oringal html file to the path outlined above
with open(path, 'w') as f:
    f.write(resp.text)

<br>

### Using regular expressions to select lines we want to view in df.

***

In [7]:
#regular expression for 2021 cao. The r at the start of the string below means raw. It prevents the backslash from being interpreted.

re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

 This is a regular expression here. It is looking to fully match a pattern. The first part [A-Z] is saying return any uppercase characters from a-z, the second part is a quantifier and it states that there are two digits(this is because the code names for the courses all start with two letter digts) The third part is a number 0-9 (all the college courses follow the two letter digits with numbers and this is folloowed by a quantifier of 3.

The line.decode part is to allow jupyter to read the lines as strings as opposed to bytes. Without this there would be an error.

<br>
Loop through the lines of the response


In [8]:
#create the file path for the csv
path = 'data/cao2021_csv_' + nowstr + '.csv'

In [9]:
 #count lines so we can know how many lines we have processed.
no_lines = 0

#open csv file
with open('data/cao2021_csv_', 'w') as f:
    # Write a header row.
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through lines of the response.
    for line in resp.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

Total number of lines is 949.


In [10]:
df2021 = pd.read_csv('data/cao2021_csv_', encoding='cp1252')

In [11]:
df2021

,code,title,pointsR1,pointsR2
0,AL801,Software Design for Virtual Reality and Gaming,300,NaN
1,AL802,Software Design in Artificial Intelligence for...,313,NaN
2,AL803,Software Design for Mobile Apps and Connected ...,350,NaN
3,AL805,Computer Engineering for Network Infrastructure,321,NaN
4,AL810,Quantity Surveying,328,NaN
...,...,...,...,...
944,WD211,Creative Computing,270,NaN
945,WD212,Recreation and Sport Management,262,NaN
946,WD230,Mechanical and Manufacturing Engineering,230,230
947,WD231,Early Childhood Care and Education,266,NaN




<br>




## 2020 Points

http://www.cao.ie/index.php?page=points&p=2020
***

<br>



In [13]:
pip install xlrd

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


I previously had an error message when I tried running the code below. I searched how to fix the error message and one of the solutions was to install xlrd. This worked but I realised that I needed to also download the latest python version to avoid future errors. Hopefully everthing runs smoothly from here out ;)

### Save original file

In [14]:
#file path for original data
path = 'data/cao2020_' + nowstr + '.xlsx'

In [15]:
# save the original file to disk

urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path)

('data/cao2020_20211125_124606.xlsx',
 <http.client.HTTPMessage at 0x7fef3d1ac340>)

<br>

#### Load Spreadsheet using pandas


***

In [16]:
#download and parse the excel spreadsheet
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows=10)

In [17]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# spot check a random row
df2020.iloc[444]

CATEGORY (i.e.ISCED description)                             Health
COURSE TITLE                           Health and Physical Activity
COURSE CODE2                                                  DK880
R1 POINTS                                                       328
R1 Random *                                                     NaN
R2 POINTS                                                       NaN
R2 Random*                                                      NaN
EOS                                                             328
EOS Random *                                                    NaN
EOS Mid-point                                                   389
LEVEL                                                             8
HEI                                 Dundalk Institute of Technology
Test/Interview #                                                NaN
avp                                                             NaN
v                                               

In [19]:
# spot check the last row.
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

In [20]:
#file path for pandas data
path = 'data/cao2020_' + nowstr + '.csv'

In [21]:
# save pandas dataframe to disk.
df2020.to_csv(path)

<br>

## 2019 Points

http://www.cao.ie/index.php?page=points&p=2019
***

In [22]:
df2019 = pd.read_csv('data/cao2019_November10th_1618', sep='\t')

In [23]:
df2019

,Course Code,Course,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,AL802,Software Design with Cloud Computing,301,306.0
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,AL805,Network Management and Cloud Infrastructure,329,442.0
4,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
925,WD200,Arts (options),221,296.0
926,WD210,Software Systems Development,271,329.0
927,WD211,Creative Computing,275,322.0
928,WD212,Recreation and Sport Management,274,311.0


<br>




## Concatinate and Join

In [24]:
courses2021 = df2021[['code', 'title']]
courses2021

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
944,WD211,Creative Computing
945,WD212,Recreation and Sport Management
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [25]:
courses2020 = df2020[['COURSE CODE2','COURSE TITLE']]
courses2020.columns = ['code', 'title']
courses2020

,code,title
0,AC120,International Business
1,AC137,Liberal Arts
2,AD101,"First Year Art & Design (Common Entry,portfolio)"
3,AD102,Graphic Design and Moving Image Design (portfo...
4,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
1459,WD208,Manufacturing Engineering
1460,WD210,Software Systems Development
1461,WD211,Creative Computing
1462,WD212,Recreation and Sport Management


In [26]:
allcourses = pd.concat([courses2021, courses2020], ignore_index=True)
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2408,WD208,Manufacturing Engineering
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [27]:
allcourses.sort_values('code')

,code,title
175,AC120,International Business
949,AC120,International Business
950,AC137,Liberal Arts
176,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
...,...,...
2411,WD212,Recreation and Sport Management
2412,WD230,Mechanical and Manufacturing Engineering
946,WD230,Mechanical and Manufacturing Engineering
947,WD231,Early Childhood Care and Education


In [28]:
allcourses.loc[2411]['title']

'Recreation and Sport Management'

In [29]:
allcourses.loc[950]['title']

'Liberal Arts'

In [30]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated()]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
952,AD102,Graphic Design and Moving Image Design (portfo...
955,AD204,Fine Art (portfolio)
956,AD211,Fashion Design (portfolio)
...,...,...
2404,WD200,Arts (options)
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [31]:
# Returns a copy of the data frame with duplciates removed.
allcourses.drop_duplicates()

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
2403,WD195,Architectural and Building Information Modelli...
2405,WD205,Molecular Biology with Biopharmaceutical Science
2406,WD206,Electronic Engineering
2407,WD207,Mechanical Engineering


In [32]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated(subset=['code'])]

,code,title
949,AC120,International Business
950,AC137,Liberal Arts
951,AD101,"First Year Art & Design (Common Entry,portfolio)"
952,AD102,Graphic Design and Moving Image Design (portfo...
953,AD103,Textile & Surface Design and Jewellery & Objec...
...,...,...
2404,WD200,Arts (options)
2409,WD210,Software Systems Development
2410,WD211,Creative Computing
2411,WD212,Recreation and Sport Management


In [33]:
# Returns a copy of the data frame with duplciates removed - based only on code.
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [34]:
allcourses

,code,title
0,AL801,Software Design for Virtual Reality and Gaming
1,AL802,Software Design in Artificial Intelligence for...
2,AL803,Software Design for Mobile Apps and Connected ...
3,AL805,Computer Engineering for Network Infrastructure
4,AL810,Quantity Surveying
...,...,...
1512,WD188,Applied Health Care
1513,WD205,Molecular Biology with Biopharmaceutical Science
1514,WD206,Electronic Engineering
1515,WD207,Mechanical Engineering


<br>


## Join the points

In [35]:
# Set the index to the code column.
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

,title,points_r1_2021,points_r2_2021
code,,,
AL801,Software Design for Virtual Reality and Gaming,300,NaN
AL802,Software Design in Artificial Intelligence for...,313,NaN
AL803,Software Design for Mobile Apps and Connected ...,350,NaN
AL805,Computer Engineering for Network Infrastructure,321,NaN
AL810,Quantity Surveying,328,NaN
...,...,...,...
WD211,Creative Computing,270,NaN
WD212,Recreation and Sport Management,262,NaN
WD230,Mechanical and Manufacturing Engineering,230,230


In [36]:
# Set the index to the code column.
allcourses.set_index('code', inplace=True)

In [37]:
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

,title,points_r1_2021
code,,
AL801,Software Design for Virtual Reality and Gaming,300
AL802,Software Design in Artificial Intelligence for...,313
AL803,Software Design for Mobile Apps and Connected ...,350
AL805,Computer Engineering for Network Infrastructure,321
AL810,Quantity Surveying,328
...,...,...
WD188,Applied Health Care,NaN
WD205,Molecular Biology with Biopharmaceutical Science,NaN
WD206,Electronic Engineering,NaN


In [38]:
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_r1_2020']
df2020_r1

,code,points_r1_2020
0,AC120,209
1,AC137,252
2,AD101,#+matric
3,AD102,#+matric
4,AD103,#+matric
...,...,...
1459,WD208,188
1460,WD210,279
1461,WD211,271
1462,WD212,270


In [39]:
# Set the index to the code column.
df2020_r1.set_index('code', inplace=True)
df2020_r1

,points_r1_2020
code,
AC120,209
AC137,252
AD101,#+matric
AD102,#+matric
AD103,#+matric
...,...
WD208,188
WD210,279
WD211,271


In [40]:
# Join 2020 points to allcourses.
allcourses = allcourses.join(df2020_r1)
allcourses

,title,points_r1_2021,points_r1_2020
code,,,
AL801,Software Design for Virtual Reality and Gaming,300,303
AL802,Software Design in Artificial Intelligence for...,313,332
AL803,Software Design for Mobile Apps and Connected ...,350,337
AL805,Computer Engineering for Network Infrastructure,321,333
AL810,Quantity Surveying,328,319
...,...,...,...
WD188,Applied Health Care,NaN,201
WD205,Molecular Biology with Biopharmaceutical Science,NaN,228
WD206,Electronic Engineering,NaN,179


<br> 

# End
***